<h1 style="color:#2E86C1;">Final Project — Family Income & Expenditure Analysis</h1>
<hr>
<h3>Analysts: Carl Kien Carabido & Marianne Mae Capuno</h3>
<p>

In [1]:
# 0. Imports (minimal)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.metrics import r2_score, mean_squared_error
import math
import joblib
import warnings
warnings.filterwarnings("ignore")

DATA_PATH = Path('datasets\\Family Income and Expenditure.csv')

In [2]:
# 1. Load dataset and quick overview
df = pd.read_csv(DATA_PATH)
df.head()

,Total Household Income,Region,Total Food Expenditure,Main Source of Income,Agricultural Household indicator,Bread and Cereals Expenditure,Total Rice Expenditure,Meat Expenditure,Total Fish and marine products Expenditure,Fruit Expenditure,...,Number of Refrigerator/Freezer,Number of Washing Machine,Number of Airconditioner,"Number of Car, Jeep, Van",Number of Landline/wireless telephones,Number of Cellular phone,Number of Personal Computer,Number of Stove with Oven/Gas Range,Number of Motorized Banca,Number of Motorcycle/Tricycle
0,480332,CAR,117848,Wage/Salaries,0,42140,38300,24676,16806,3325,...,1,1,0,0,0,2,1,0,0,1
1,198235,CAR,67766,Wage/Salaries,0,17329,13008,17434,11073,2035,...,0,1,0,0,0,3,1,0,0,2
2,82785,CAR,61609,Wage/Salaries,1,34182,32001,7783,2590,1730,...,0,0,0,0,0,0,0,0,0,0
3,107589,CAR,78189,Wage/Salaries,0,34030,28659,10914,10812,690,...,0,0,0,0,0,1,0,0,0,0
4,189322,CAR,94625,Wage/Salaries,0,34820,30167,18391,11309,1395,...,1,0,0,0,0,3,0,0,0,1


### Note
We will use these variables:
- Target: **Total Household Income**
- Type of Household (Single vs Extended vs Others)
- **Total Number of Family members**
- **Household Head Highest Grade Completed**
- **Total Food Expenditure**
- **Education Expenditure**
- **Total number of family members employed**
- **Region**

In [3]:
# 2. Basic column check and keep only columns we need
cols_needed = [
    'Total Household Income',
    'Type of Household',
    'Total Number of Family members',
    'Household Head Highest Grade Completed',
    'Total Food Expenditure',
    'Education Expenditure',
    'Total number of family members employed',
    'Region'
]
# Keep those that exist
cols_use = [c for c in cols_needed if c in df.columns]
print("Using columns:", cols_use)
df_small = df[cols_use].copy()
df_small.info()

Using columns: ['Total Household Income', 'Type of Household', 'Total Number of Family members', 'Household Head Highest Grade Completed', 'Total Food Expenditure', 'Education Expenditure', 'Total number of family members employed', 'Region']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41544 entries, 0 to 41543
Data columns (total 8 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Total Household Income                   41544 non-null  int64 
 1   Type of Household                        41544 non-null  object
 2   Total Number of Family members           41544 non-null  int64 
 3   Household Head Highest Grade Completed   41544 non-null  object
 4   Total Food Expenditure                   41544 non-null  int64 
 5   Education Expenditure                    41544 non-null  int64 
 6   Total number of family members employed  41544 non-null  int64 
 7   Region                 

### Short explanation
If any of the listed columns are missing, the code will continue using available columns. The notebook uses simple logic so it's robust to small differences in names/data.

In [4]:
# 3. Map education to simple ordered categories (ordinal)
def map_education(txt):
    if pd.isna(txt):
        return 'Unknown'
    s = str(txt).lower()
    if any(k in s for k in ['master','phd','graduate','post','doctor']):
        return 'Graduate'
    if any(k in s for k in ['college','degree','vocational','training','teacher','undergrad','tertiary','programs']):
        return 'Vocational/College'
    if any(k in s for k in ['high school','secondary','senior high','junior high','senior']):
        return 'HighSchool'
    if any(k in s for k in ['elementary','grade 1','grade 2','grade 3','grade 4','grade 5','grade 6']):
        return 'Elementary'
    if any(k in s for k in ['none','no formal']):
        return 'NoFormal'
    return 'Other'

if 'Household Head Highest Grade Completed' in df_small.columns:
    df_small['edu_level'] = df_small['Household Head Highest Grade Completed'].apply(map_education)
else:
    df_small['edu_level'] = 'Unknown'

edu_order = ['NoFormal','Elementary','HighSchool','Vocational/College','Graduate','Other','Unknown']
edu_rank = {k:i for i,k in enumerate(edu_order)}
df_small['edu_ordinal'] = df_small['edu_level'].map(edu_rank).fillna(len(edu_order)-1).astype(int)

### Explanation
We convert the many text labels into a small set of ordered categories so we can:
- compute group medians easily, and
- use a simple numeric ordinal for regression.

In [5]:
# 4. Type of Household, size, education, expenditures
# A: Type of Household summary
if 'Type of Household' in df_small.columns:
    th = df_small.groupby('Type of Household')['Total Household Income'].agg(['count','median','mean']).sort_values('median', ascending=False)
    th['median_income_per_capita'] = df_small.groupby('Type of Household').apply(lambda g: (g['Total Household Income'] / g['Total Number of Family members']).median() if 'Total Number of Family members' in g.columns else np.nan)
    print("Type of Household summary (median total income, median per-capita):")
    display(th)
    # Short plain-English comment
    print("Note: Extended families typically show higher total median income but per-capita may be lower.")
else:
    print("Type of Household not found.")

Type of Household summary (median total income, median per-capita):


,count,median,mean,median_income_per_capita
Type of Household,,,,
Two or More Nonrelated Persons/Members,167,254350.0,364741.532934,49528.571429
Extended Family,12932,217317.0,308349.946721,39371.875000
Single Family,28445,146320.0,219228.541677,40668.666667


Note: Extended families typically show higher total median income but per-capita may be lower.


In [6]:
# B: Median income by household size
if 'Total Number of Family members' in df_small.columns:
    size_med = df_small.groupby('Total Number of Family members')['Total Household Income'].median().sort_index()
    print("Median income by household size (first 12 sizes):")
    display(size_med.head(12))
    print("Note: Income tends to increase with household size (total income), but per-capita may not.")
else:
    print("Household size column not found.")

Median income by household size (first 12 sizes):


Total Number of Family members
1      82680.0
2     124486.0
3     156241.0
4     167761.0
5     174610.0
6     176319.0
7     194468.0
8     187830.0
9     223359.0
10    218768.0
11    284099.0
12    265669.5
Name: Total Household Income, dtype: float64

Note: Income tends to increase with household size (total income), but per-capita may not.


In [7]:
# C: Median income by education level
edu_med = df_small.groupby('edu_level')['Total Household Income'].median().reindex(edu_order)
print("Median income by education level:")
display(edu_med)
print("Note: Vocational/College and Graduate usually have higher median incomes.")

Median income by education level:


edu_level
NoFormal                   NaN
Elementary            114200.0
HighSchool            146322.5
Vocational/College    328334.5
Graduate              165494.0
Other                  94350.0
Unknown                    NaN
Name: Total Household Income, dtype: float64

Note: Vocational/College and Graduate usually have higher median incomes.


In [8]:
# D: Simple expenditure summaries
for col in ['Total Food Expenditure','Education Expenditure']:
    if col in df_small.columns:
        print(f"\n{col} — national median and mean:")
        display(df_small[col].agg(['median','mean','sum']))
    else:
        print(f"{col} not present.")


Total Food Expenditure — national median and mean:


median    7.298550e+04
mean      8.509916e+04
sum       3.535359e+09
Name: Total Food Expenditure, dtype: float64


Education Expenditure — national median and mean:


median    8.800000e+02
mean      7.473500e+03
sum       3.104791e+08
Name: Education Expenditure, dtype: float64

### Interpretation
We look at income by household type, size, education, and two key expenditures (food & education).  
Next we compare correlations at national vs regional levels.

In [9]:
# 5. NATIONAL correlations (simple)
vars_for_corr = []
if 'Total Household Income' in df_small.columns:
    vars_for_corr.append('Total Household Income')
if 'edu_ordinal' in df_small.columns:
    vars_for_corr.append('edu_ordinal')
if 'Total Number of Family members' in df_small.columns:
    vars_for_corr.append('Total Number of Family members')
if 'Total number of family members employed' in df_small.columns:
    vars_for_corr.append('Total number of family members employed')
if 'Total Food Expenditure' in df_small.columns:
    vars_for_corr.append('Total Food Expenditure')
if 'Education Expenditure' in df_small.columns:
    vars_for_corr.append('Education Expenditure')

print("Variables used for correlation (national):", vars_for_corr)
national_corr = df_small[vars_for_corr].corr()
print("National correlation matrix:")
display(national_corr)
print("Short note: we will now compute the same correlations per region and compare.")

Variables used for correlation (national): ['Total Household Income', 'edu_ordinal', 'Total Number of Family members', 'Total number of family members employed', 'Total Food Expenditure', 'Education Expenditure']
National correlation matrix:


,Total Household Income,edu_ordinal,Total Number of Family members,Total number of family members employed,Total Food Expenditure,Education Expenditure
Total Household Income,1.000000,0.094810,0.145149,0.211099,0.663660,0.439593
edu_ordinal,0.094810,1.000000,-0.029043,0.008984,0.110367,0.055021
Total Number of Family members,0.145149,-0.029043,1.000000,0.405751,0.418187,0.093491
Total number of family members employed,0.211099,0.008984,0.405751,1.000000,0.342087,0.040593
Total Food Expenditure,0.663660,0.110367,0.418187,0.342087,1.000000,0.374898
Education Expenditure,0.439593,0.055021,0.093491,0.040593,0.374898,1.000000


Short note: we will now compute the same correlations per region and compare.


In [10]:
# 6. REGIONAL correlations and simple comparison table
if 'Region' in df_small.columns:
    regions = df_small['Region'].dropna().unique().tolist()
    # build a region vs variable correlation table for income vs key predictors
    rows = []
    for r in regions:
        sub = df_small[df_small['Region']==r]
        if len(sub) < 50:
            # skip very small regions to avoid noisy correlations, but keep note
            continue
        row = {'Region': r, 'N': len(sub)}
        # compute correlations with income
        for v in vars_for_corr:
            if v == 'Total Household Income': continue
            try:
                row[f'corr_income__{v}'] = sub['Total Household Income'].corr(sub[v])
            except Exception:
                row[f'corr_income__{v}'] = np.nan
        rows.append(row)
    region_corr_df = pd.DataFrame(rows).set_index('Region').sort_values('N', ascending=False)
    print("Regional correlations (income vs variables). Regions with <50 obs skipped.")
    display(region_corr_df.head(20))
    # Quick textual comparison: show top differences for education vs income across regions
    if 'corr_income__edu_ordinal' in region_corr_df.columns:
        print("\nRegions with strongest/weakest edu-income correlation:")
        display(region_corr_df['corr_income__edu_ordinal'].dropna().sort_values(ascending=False).head(8))
        display(region_corr_df['corr_income__edu_ordinal'].dropna().sort_values(ascending=True).head(8))
else:
    print("Region column not found — cannot do regional comparison.")


Regional correlations (income vs variables). Regions with <50 obs skipped.


,N,corr_income__edu_ordinal,corr_income__Total Number of Family members,corr_income__Total number of family members employed,corr_income__Total Food Expenditure,corr_income__Education Expenditure
Region,,,,,,
IVA - CALABARZON,4162,0.042054,0.198714,0.244806,0.743879,0.492674
NCR,4130,-0.022588,0.205724,0.286686,0.665682,0.390987
III - Central Luzon,3237,0.052286,0.231016,0.176945,0.628140,0.435065
VI - Western Visayas,2851,0.060547,0.133174,0.082791,0.504505,0.434163
VII - Central Visayas,2541,0.140542,0.176558,0.217692,0.497202,0.441638
V - Bicol Region,2472,0.094856,0.124488,0.198359,0.737610,0.505648
XI - Davao Region,2446,0.122570,0.163480,0.225675,0.714312,0.537198
I - Ilocos Region,2348,0.033079,0.250974,0.122267,0.727580,0.440075
VIII - Eastern Visayas,2337,0.106813,0.089303,0.144788,0.625145,0.402644



Regions with strongest/weakest edu-income correlation:


Region
X - Northern Mindanao        0.165433
Caraga                       0.158436
VII - Central Visayas        0.140542
XI - Davao Region            0.122570
IX - Zasmboanga Peninsula    0.111551
VIII - Eastern Visayas       0.106813
V - Bicol Region             0.094856
CAR                          0.081310
Name: corr_income__edu_ordinal, dtype: float64

Region
NCR                    -0.022588
 ARMM                   0.023324
I - Ilocos Region       0.033079
IVA - CALABARZON        0.042054
III - Central Luzon     0.052286
XII - SOCCSKSARGEN      0.058938
VI - Western Visayas    0.060547
IVB - MIMAROPA          0.063240
Name: corr_income__edu_ordinal, dtype: float64

### Short explanation
This table shows the correlation between Total Household Income and each predictor **within each region**.

In [11]:
# 7. Simple per-region linear regression (income ~ edu_ordinal) for a few top regions
# Keep it simple: only run regression for regions with enough observations
from sklearn.linear_model import LinearRegression
reg_results = []
if 'Region' in df_small.columns:
    for r in region_corr_df.index[:8]:  # limit to top 8 region rows shown
        sub = df_small[df_small['Region']==r]
        # require at least 100 rows to run regression
        if len(sub) < 100:
            continue
        X = sub[['edu_ordinal']].copy()
        y = sub['Total Household Income'].copy()
        # log-transform to handle skew — simple rule: use log1p
        y_log = np.log1p(y)
        model = LinearRegression()
        model.fit(X, y_log)
        pred = model.predict(X)
        r2 = r2_score(y_log, pred)
        coef = float(model.coef_[0])
        reg_results.append({'Region': r, 'N': len(sub), 'coef_edu': coef, 'r2_log': r2})
    reg_df = pd.DataFrame(reg_results).set_index('Region').sort_values('N', ascending=False)
    print("Per-region simple regression results (income_log ~ edu_ordinal):")
    display(reg_df)
else:
    print("Region missing, skipping per-region regression.")


Per-region simple regression results (income_log ~ edu_ordinal):


,N,coef_edu,r2_log
Region,,,
IVA - CALABARZON,4162,0.060696,0.008294
NCR,4130,0.000887,0.000001
III - Central Luzon,3237,0.060805,0.009222
VI - Western Visayas,2851,0.064524,0.013150
VII - Central Visayas,2541,0.139174,0.045638
V - Bicol Region,2472,0.070419,0.015957
XI - Davao Region,2446,0.111926,0.036060
I - Ilocos Region,2348,0.047989,0.005307


### Interpretation
We ran a simple regression per region to see whether education's effect on income (slope) changes by region.  
Compare `coef_edu` and `r2_log` across regions to see where education matters more.

In [12]:
# 8. Simple national predictive models (kept minimal)
# Features: edu_ordinal, Total Number of Family members, employed count, Type of Household dummies
features = ['edu_ordinal']
if 'Total Number of Family members' in df_small.columns:
    features.append('Total Number of Family members')
if 'Total number of family members employed' in df_small.columns:
    features.append('Total number of family members employed')

X = df_small[features].copy()
# one-hot for type of household if present
if 'Type of Household' in df_small.columns:
    oh = pd.get_dummies(df_small['Type of Household'], prefix='house', drop_first=True)
    X = pd.concat([X, oh], axis=1)

X = X.fillna(X.median())
y = df_small['Total Household Income'].fillna(df_small['Total Household Income'].median())

# log target for better behavior
y_log = np.log1p(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)

# Linear regression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
r2_lr = r2_score(y_test, y_pred_lr)
rmse_lr = math.sqrt(mean_squared_error(y_test, y_pred_lr))
# Decision tree shallow
dt = DecisionTreeRegressor(max_depth=3, random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
r2_dt = r2_score(y_test, y_pred_dt)
rmse_dt = math.sqrt(mean_squared_error(y_test, y_pred_dt))

# Report results (also compute RMSE back to original scale for intuition)
y_test_orig = np.expm1(y_test)
y_pred_lr_orig = np.expm1(y_pred_lr)
y_pred_dt_orig = np.expm1(y_pred_dt)
rmse_lr_orig = math.sqrt(mean_squared_error(y_test_orig, y_pred_lr_orig))
rmse_dt_orig = math.sqrt(mean_squared_error(y_test_orig, y_pred_dt_orig))

print("Model summary (simple, national):")
print(f"Linear R2 (log-target): {r2_lr:.3f}  | RMSE (orig scale): ₱{int(rmse_lr_orig):,}")
print(f"DecisionTree R2 (log-target): {r2_dt:.3f} | RMSE (orig scale): ₱{int(rmse_dt_orig):,}")

# Coefficients (linear)
coef_df = pd.DataFrame({'feature': X.columns, 'coef': lr.coef_}).sort_values('coef', key=lambda s: s.abs(), ascending=False)
print("\nTop linear coefficients (absolute):")
display(coef_df.head(10))

# Decision tree importance and rules
imp_df = pd.DataFrame({'feature': X.columns, 'importance': dt.feature_importances_}).sort_values('importance', ascending=False)
print("\nDecision tree feature importances:")
display(imp_df)
print("\nDecision tree rules (shallow):")
try:
    print(export_text(dt, feature_names=list(X.columns), decimals=2))
except Exception:
    print("(Could not export tree rules.)")

Model summary (simple, national):
Linear R2 (log-target): 0.155  | RMSE (orig scale): ₱265,713
DecisionTree R2 (log-target): 0.275 | RMSE (orig scale): ₱249,023

Top linear coefficients (absolute):


,feature,coef
3,house_Single Family,-0.201936
2,Total number of family members employed,0.147173
0,edu_ordinal,0.098465
4,house_Two or More Nonrelated Persons/Members,0.084074
1,Total Number of Family members,0.043764



Decision tree feature importances:


,feature,importance
0,edu_ordinal,0.679765
1,Total Number of Family members,0.218314
2,Total number of family members employed,0.101921
3,house_Single Family,0.000000
4,house_Two or More Nonrelated Persons/Members,0.000000



Decision tree rules (shallow):
|--- edu_ordinal <= 2.50
|   |--- Total Number of Family members <= 2.50
|   |   |--- Total Number of Family members <= 1.50
|   |   |   |--- value: [11.01]
|   |   |--- Total Number of Family members >  1.50
|   |   |   |--- value: [11.42]
|   |--- Total Number of Family members >  2.50
|   |   |--- Total number of family members employed <= 2.50
|   |   |   |--- value: [11.81]
|   |   |--- Total number of family members employed >  2.50
|   |   |   |--- value: [12.19]
|--- edu_ordinal >  2.50
|   |--- edu_ordinal <= 3.50
|   |   |--- Total number of family members employed <= 1.50
|   |   |   |--- value: [12.53]
|   |   |--- Total number of family members employed >  1.50
|   |   |   |--- value: [13.02]
|   |--- edu_ordinal >  3.50
|   |   |--- Total Number of Family members <= 2.50
|   |   |   |--- value: [11.59]
|   |   |--- Total Number of Family members >  2.50
|   |   |   |--- value: [12.12]



### Short comment
We kept models intentionally minimal and easy to interpret. Linear coefficients are on the log-target scale.

<h3 style="color:#117A65;">INSIGHT 6 — National correlation between income and education is weak (r = 0.095), but strong in specific regions</h3>
<blockquote> <b>Evidence:</b> Notebook results: <br>• National corr = 0.095 <br>• Region X = 0.165 <br>• Caraga = 0.158 <br>• Region VII = 0.141 <br>• NCR = –0.023 </blockquote>
<p><b>Insight:</b> Education’s impact on income depends on the region. Outside NCR, schooling greatly improves income. In NCR, income depends more on location and job demand, not education level.</p> <p><b>Analogy:</b> Education is like fertilizer — it works better in some soils (regions) than others.</p>
<p><b>Recommendation:</b> Create region-specific education-to-employment programs instead of one nationwide template.</p>
<hr>

<h3 style="color:#117A65;">INSIGHT 7 — Food expenditure strongly correlates with income (r ≈ 0.664)</h3>
<blockquote> <b>Evidence:</b> Correlation matrix in Notebook shows Total Food Expenditure has one of the highest correlations with income (r = 0.664). </blockquote>
<p><b>Insight:</b> Higher-income families consistently spend more on food quality, quantity, and variety.</p> <p><b>Analogy:</b> Food spending is like a mirror — it reflects a household’s financial health.</p>
<p><b>Recommendation:</b> Use food expenditure as a key poverty-identification metric.</p>
<hr>

<h3 style="color:#117A65;">INSIGHT 8 — Decision Tree shows education is the strongest predictor of income (feature importance ≈ 0.68)</h3>
<blockquote> <b>Evidence:</b> Decision Tree (depth=3) feature importances: <br>• Education = 0.68 <br>• Household Size = 0.22 <br>• Employed Members = 0.10 </blockquote>
<p><b>Insight:</b> Among simple predictors, education has the biggest influence on household income growth.</p> <p><b>Analogy:</b> Education is the “main road” to higher income — the other variables are just side streets.</p>
<p><b>Recommendation:</b> Prioritize education improvements to boost long-term income potential.</p>
<hr>

<h3 style="color:#117A65;">INSIGHT 9 — Linear Regression shows employed family members increase income the most</h3>
<blockquote> <b>Evidence:</b> Notebook regression: “Total number of family members employed” has the strongest positive coefficient in the log-income model. </blockquote>
<p><b>Insight:</b> More working members consistently increase total household income, regardless of household size.</p>
<p><b>Analogy:</b> It’s like rowing a boat — more rowers push the boat faster and farther.</p>
<p><b>Recommendation:</b> Improve employability programs for all working-age members, not just heads of households.</p>
<hr>

<h3 style="color:#117A65;">INSIGHT 10 — Education predicts income much better in rural regions than NCR</h3>
<blockquote> <b>Evidence:</b> Per-region regressions show NCR has near-zero R², while regions VII, X, and Caraga show much higher coefficients and R² values. </blockquote>
<p><b>Insight:</b> In rural regions, education significantly boosts income. In NCR, job demand and industry density weaken the role of education.</p> <p><b>Analogy:</b> In rural areas, education is a “ladder.” In NCR, it's just one tool among many in a crowded toolbox.</p>
<p><b>Recommendation:</b> Allocate more scholarships and educational funding to rural areas where the income returns are highest.</p>
<hr>